# Frontal Gaze classifier

### Dataset autosplit

In [1]:
import tensorflow as tf
import keras
from keras.models import Model
from keras import layers
from keras.layers import Dense
import matplotlib.pyplot as plt

2025-04-16 17:47:24.206051: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 17:47:24.296065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 17:47:24.319196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 17:47:24.325962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 17:47:24.410519: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory='dataset/train',
    labels='inferred',
    label_mode='categorical',
    class_names=['close_look', 'forward_look', 'left_look', 'right_look'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 14320 files belonging to 4 classes.
Using 11456 files for training.


I0000 00:00:1744818448.350004   29162 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744818448.554921   29162 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744818448.554966   29162 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744818448.558495   29162 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744818448.558533   29162 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
val_ds = keras.preprocessing.image_dataset_from_directory(
    directory='dataset/train',
    labels='inferred',
    label_mode='categorical',
    class_names=['close_look', 'forward_look', 'left_look', 'right_look'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 14320 files belonging to 4 classes.
Using 2864 files for validation.


In [4]:
test_ds = keras.preprocessing.image_dataset_from_directory(
    directory='dataset/test',
    labels='inferred',
    label_mode='categorical',
    class_names=['close_look', 'forward_look', 'left_look', 'right_look'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 40 files belonging to 4 classes.


In [5]:
model = keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
    )

In [6]:
# Create a layer where input is the output of the second last layer
output = Dense(4, activation='softmax', name='predictions')(model.layers[-2].output)

# Then create the corresponding model
model = Model(model.input, output)

In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 4,257,868 (16.24 MB)

 Trainable params: 4,235,980 (16.16 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [8]:
epochs = 250

callbacks = [
    # logging
    keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, write_images=False, update_freq="epoch",),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)  # Monitora 'val_loss' per fermarsi prima

]

model.compile(
    optimizer=keras.optimizers.SGD(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [9]:
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

model.save("model.keras")

Epoch 1/250


I0000 00:00:1744818453.612298   32094 service.cc:146] XLA service 0x7f3010003070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744818453.612394   32094 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2025-04-16 17:47:33.753439: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-16 17:47:34.065975: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


  1/358 ━━━━━━━━━━━━━━━━━━━━ 1:16:46 13s/step - accuracy: 0.4688 - loss: 2.4107

I0000 00:00:1744818464.554147   32094 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


358/358 ━━━━━━━━━━━━━━━━━━━━ 29s 44ms/step - accuracy: 0.7393 - loss: 0.8537 - val_accuracy: 0.8593 - val_loss: 0.3574
Epoch 2/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9520 - loss: 0.1346 - val_accuracy: 0.9581 - val_loss: 0.1302
Epoch 3/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9733 - loss: 0.0854 - val_accuracy: 0.9571 - val_loss: 0.1217
Epoch 4/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9803 - loss: 0.0610 - val_accuracy: 0.9756 - val_loss: 0.0829
Epoch 5/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9897 - loss: 0.0409 - val_accuracy: 0.9822 - val_loss: 0.0630
Epoch 6/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9896 - loss: 0.0373 - val_accuracy: 0.9797 - val_loss: 0.0750
Epoch 7/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9911 - loss: 0.0309 - val_accuracy: 0.9804 - val_loss: 0.0601
Epoch 8/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9929 - loss: 0.0237 - val_

In [10]:
model_trained = keras.models.load_model('./model.keras')

model_trained.evaluate(
    x=test_ds,
    y=None,
    verbose=True,
    sample_weight=None,
    steps=None,
    callbacks=None,
    return_dict=False,
)

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.8146 - loss: 0.7168  


[0.705793023109436, 0.800000011920929]

In [11]:
import numpy as np
img_path = "./dataset/test/close_look/eye_closed (5).png"
img = keras.utils.load_img(img_path, target_size=(224, 224))
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
img_array = img_array / 255.0

predictions = model_trained.predict(img_array)
predicted_class = np.argmax(predictions[0])
confidence = np.max(predictions[0])
print(f"{img_path}: {predicted_class} ({confidence:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
./dataset/test/close_look/eye_closed (5).png: 1 (0.97)
